#### **CS5480: Assignment 3 - Programming Questions**

#### **Instructions**
1. Use Python 3.x to run this notebook
2. Write your code only in between the lines 'YOUR CODE STARTS HERE' and 'YOUR CODE ENDS HERE'.
you should not change anything else in the code cells, if you do, the answers you are supposed to get at the end of this assignment might be wrong.
3. Read documentation of each function carefully.
4. All the Best!

### Programming Q3. Densenet from scratch

In this question, you'll have to code Densenet from scratch (we have provided a lot of starter code), this'll help you get a hold on how to code an architecture with bottleneck layers.

It's suggested you first briefly understand how the Densenet architecture is defined originally before you start with this question. We do take inspiration from the original Pytorch implementation, but if you try peeking into the original source code in the library, it'll confuse you more than helping!

**Sidenote:** As this assignment is mainly focused on learning things, we train the models only for a small number of epochs and don't focus on hyper-parameter tuning. When you start using deep learning in real-world applications and competitions, hyper-parameter tuning plays a decent role!

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision
import torch.nn.functional as F
import timeit
import unittest
import math
from torch.autograd import Variable

## Please DONOT remove these lines. 
torch.manual_seed(2021)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(2021)

In [ ]:
# check availability of GPU and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# define a set of transforms for preparing the dataset
transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=8), 
        transforms.RandomHorizontalFlip(), # flip the image horizontally (use pytorch random horizontal flip)
        transforms.ToTensor(), # convert the image to a pytorch tensor
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # normalise the images with mean and std of the dataset
        ])

# define transforms for the test data: Should they be same as the one used for train? 
transform_test = transforms.Compose([                    
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])

use_cuda = torch.cuda.is_available() # if you have acess to a GPU, enable it to speed the training 

In [ ]:
# Load the CIFAR10 training, test datasets using `torchvision.datasets.cifar10`
#### YOUR CODE STARTS HERE ####
train_dataset =  
test_dataset = 
#### YOUR CODE ENDS HERE ####

In [ ]:
# create dataloaders for training and test datasets
# use a batch size of 32 and set shuffle=True for the training set
#### YOUR CODE STARTS HERE ####
train_dataloader = 
test_dataloader = 
#### YOUR CODE ENDS HERE ####

In [ ]:
class SingleLayer(nn.Module):
    def __init__(self, nChannels, outChannels):
        super(SingleLayer, self).__init__()
    #### YOUR CODE STARTS HERE ####
        # define a batchnorm2d layer with number of channels equal to nChannels
        self.bn1 = 
        # define a convolutional layer with input channels=nChannels, number of output channels=outChannels
        # kernel size of 3x3
        # use padding of 1
        # set bias to False
        self.conv1 = 
    #### YOUR CODE ENDS HERE ####

    def forward(self, x):
  
        # pass the input through, bn1->relu->conv1
        out =  self.conv1(F.relu(self.bn1(x)))
   #### YOUR CODE STARTS HERE ####
        # contactinate output with the input , try using cat function
        out = 
    #### YOUR CODE ENDS HERE ####
        return out

class Transition(nn.Module):
    def __init__(self, inChannels, outChannels):
        super(Transition, self).__init__()
    #### YOUR CODE STARTS HERE ####
        # define a batchnorm2d layer with number of channels equal to inChannels
        self.bn1 =
        # define a convolutional layer with input channels=inChannels, number of output channels=outChannels
        # kernel size of 1x1
        # set bias to False
        self.conv1 = 
    #### YOUR CODE ENDS HERE ####

    def forward(self, x):
    
        # pass the input through, bn1->relu->conv1
        out = self.conv1(F.relu(self.bn1(x)))
   #### YOUR CODE STARTS HERE ####
        # pass output through avg_pool2d with stride=2
        out = 
    #### YOUR CODE ENDS HERE ####
        return out


In [ ]:
class DenseNet(nn.Module):
    def __init__(self, growthRate, reduction, nClasses):
        super(DenseNet, self).__init__()
        # number of dense blocks
        nBlocks = 4

        inChannels = 2*growthRate
        #### YOUR CODE STARTS HERE ####
        # define a convolutional layer with input channels=3, number of output channels=inChannels
        # kernel size of 3x3
        # use padding of 1
        # set bias to False
        self.conv1 = 
        # use _make_dense (defined below) function to make a dense block of 4 blocks
        # pass inChannels, growthRate and nBlocks as parameter
        self.dense1 = 
        inChannels += nBlocks*growthRate
        OutChannels = int(math.floor(inChannels*reduction))
        # define a Transition block with input channels=inChannels, number of output channels=OutChannels
        self.trans1 = 

        inChannels = OutChannels
        # use _make_dense function to make a dense block of 4 blocks
        # pass inChannels, growthRate and nBlocks as parameter
        self.dense2 = 
        inChannels += nBlocks*growthRate
        OutChannels = int(math.floor(inChannels*reduction))
        # define a Transition block with input channels=inChannels, number of output channels=OutChannels
        self.trans2 = 

        inChannels = OutChannels
        # define a batchnorm2d layer with number of channels equal to inChannels
        self.bn1 = 
        # define a linear layer with input neurons=inChannels and output neurons=nClasses
        self.fc = 
        #### YOUR CODE ENDS HERE ####

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def _make_dense(self, inChannels, growthRate, nBlocks):
        layers = []
        for i in range(int(nBlocks)):
          layers.append(SingleLayer(inChannels, growthRate))
          inChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        # pass the input through conv1->dense1->trans1
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
     #### YOUR CODE STARTS HERE ####
        # pass the output through dense2->trans2
        out = 
        # pass the output through bn1->relu->avg_pool2d
        #flatten the output (you can use squeeze function)
        out = 
        #pass the output through fc->log_softmax
        out = 
    #### YOUR CODE ENDS HERE ####
        return out

In [ ]:
def train(epoch, net, trainLoader, optimizer):


    net.train()
    nProcessed = 0
    nTrain = len(trainLoader.dataset)
    for batch_idx, (data, target) in enumerate(trainLoader):
      
        # send the data, target to the device 
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        # flush out the gradients stored in optimizer
        optimizer.zero_grad()
        # pass the data to the model and assign the output to variable named output
        output = net(data)
        # calculate the loss (use nll_loss)
        loss = F.nll_loss(output, target)
        # do a backward pass on loss
        loss.backward()
        # update the weights
        optimizer.step()

        nProcessed += len(data)
        pred = output.data.max(1)[1] # get the index of the max log-probability
        incorrect = pred.ne(target.data).cpu().sum()
        err = 100.*incorrect/len(data)
        partialEpoch = epoch + batch_idx / len(trainLoader) - 1
        print('Train Epoch: {:.2f} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tError: {:.6f}'.format(
            partialEpoch, nProcessed, nTrain, 100. * batch_idx / len(trainLoader),
            loss.item(), err))

In [ ]:
def test(epoch, net, testLoader, optimizer):
    net.eval()
    test_loss = 0
    incorrect = 0
    for data, target in testLoader:
        # send the data, target to the device
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        # pass the image to the model and assign the output to variable named output
        output =net(data)

        test_loss += F.nll_loss(output, target).item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        incorrect += pred.ne(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(testLoader) # loss function already averages over batch size
    nTotal = len(testLoader.dataset)
    err = 100.*incorrect/nTotal
    print('\nTest set: Average loss: {:.4f}, Error: {}/{} ({:.0f}%)\n'.format(
        test_loss, incorrect, nTotal, err))


#### Question:

Report the final test accuracy displayed (Hint: It may be closer to getting a number less than 5 on a six-faced dice, than getting a number less than 4 or 6!).

In [ ]:
### YOUR CODE STARTS HERE ####
#define the Densenet model with growthRate=2, reduction=0.5 and number of classes=10
model = 
print('  + Number of params: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
#move the model to cuda
model = 
## Define SGD Optimiser with a learning rate of 0.01
optimizer = 
### YOUR CODE ENDS HERE ####
for epoch in range(1, 5):
        train(1, model, train_dataloader, optimizer)
        test(1,model, test_dataloader, optimizer)

### Programming Q4a: Simple Gradient
Visualize saliency map from Pretrained VGG16 model of a given image using "Simple Gradient" method. Find out the mean and maximum  pixel intensity value of the generated saliency map. Note that the input image has 3 channels. To derive a single class saliency value for each pixel (i, j),  take the maximum magnitude across all colour channels.

In [ ]:
#IMPORTS

import torch
import torchvision
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary
from PIL import Image

#Using VGG-16 pretrained model for image classification

model = torchvision.models.vgg16(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Opening the image
img = Image.open('input.jpg') 

# Preprocess the image
def preprocess(image, size=224):
    transform = T.Compose([
        T.Resize((size,size)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        T.Lambda(lambda x: x[None]),
    ])
    return transform(image)

# preprocess the image
X = preprocess(img)

### YOUR CODE STARTS HERE ###

# we would run the model in evaluation mode


# we need to find the gradient with respect to the input image, so we need to call requires_grad_ on it


'''
forward pass through the model to get the logit scores.
'''

logits =

# Get the index corresponding to the maximum score and the maximum score itself.
logits_max_index = 
logits_max = 

'''
backward function on logits_max performs the backward pass in the computation graph and calculates the gradient of 
score_max with respect to nodes in the computation graph
'''


'''
Saliency would be the gradient with respect to the input image now. But note that the input image has 3 channels,
R, G and B. To derive a single class saliency value for each pixel (i, j),  we take the maximum magnitude
across all colour channels.
'''
saliency_map, _ = 
print (torch.mean(saliency_map),torch.max(saliency_map))
# code to plot the saliency map as a heatmap
plt.imshow(saliency_map[0], cmap=plt.cm.hot)
plt.axis('off')
plt.show()
### YOUR CODE ENDS HERE ###

### Programming Q4b: Guided Backprop

For a given input image, find out the mean value of the negative saliency map and positive saliency map  of final convolution layer of a pretrained Alexnet using guided backprop for the class prediction corresponding to "highest logit score". 

In [ ]:
import torch
from torch.nn import ReLU
import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import models
import cv2
import numpy as np
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
def preprocess_image(img, resize_im=True):
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    #ensure or transform incoming image to PIL image
    if type(img) != Image.Image:
        try:
            img = Image.fromarray(img)
        except Exception as e:
            print("could not transform PIL_img to a PIL Image object. Please check input.")

    # Resize image
    if resize_im:
        img = img.resize((224, 224), Image.ANTIALIAS)

    im_arr = np.float32(img)
    im_arr = im_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_arr):
        im_arr[channel] /= 255
        im_arr[channel] -= mean[channel]
        im_arr[channel] /= std[channel]
    # Convert to float tensor
    im_float = torch.from_numpy(im_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_float.unsqueeze_(0)
    # Convert to Pytorch variable
    im_var = Variable(im_float, requires_grad=True)
    return im_var



### Programming Q4c: Grad-CAM

For a given input image, find out the channel index and the importance(weight) score of most important feature map (out of all the feature maps of final convolution layer of a pretrained ResNet34 model) using Grad-CAM method for the class prediction corresponding to "highest logit score". Note that Grad-CAM produces final heatmap using the weighted combination of the feature map activations, where weights corresponds to importance score.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Function
from torchvision import models


def process_image(img):
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]

    p_img = img.copy()[:, :, ::-1]
    for i in range(3):
        p_img[:, :, i] = p_img[:, :, i] - means[i]
        p_img[:, :, i] = p_img[:, :, i] / stds[i]
    p_img = np.ascontiguousarray(np.transpose(p_img, (2, 0, 1)))
    p_img = torch.from_numpy(p_img)
    p_img.unsqueeze_(0)
    input = p_img.requires_grad_(True)
    return input

class Feat_Extractor():
    """ register gradients get activations from targetted intermediate layers """
    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x


class Netout():
    """ Get network output through forward pass and get intermediate layer representation
    and gradient computation for targeted intermediate layer  """

    def __init__(self, net, feat_module, target_layers):
        self.model = net
        self.feature_module = feat_module
        self.feature_extractor = Feat_Extractor(self.feature_module, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations = []
        for name, module in self.model._modules.items():
            if module == self.feature_module:
                target_activations, x = self.feature_extractor(x)

            elif "avgpool" in name.lower():
                x = module(x)
                x = x.view(x.size(0),-1)
    
            else:
                x = module(x)
        
        return target_activations, x


#visualize heatmap on input image
def visualize(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    img = np.uint8(255 * cam)
    plt.imshow(img)


class GradientCam:
    def __init__(self, model, feature_module, target_layer_, cuda):
        self.model = model
        self.feature_module = feature_module
        self.model.eval()
        self.cuda = cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = Netout(self.model, self.feature_module, target_layer_)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        
        ''' This function should Return weights corresponding to each feature map of last convolution layer.
        Note that, linear combination of such weights with last conv layer feature map finally 
        produce the explanation map'''

        ### YOUR CODE STARTS HERE ###
        if self.cuda:
            features, output = 
        else:
            features, output = 

        if index == None:
            index = 

        one_hot = 
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)

        self.feature_module.zero_grad()
        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = 

        target = 
        target = 

        weights = 
        class_activation_map = 

        for i, w in enumerate(weights):
            class_activation_map += 

        class_activation_map = np.maximum(class_activation_map, 0)
        class_activation_map = cv2.resize(class_activation_map, input.shape[2:])
        class_activation_map = class_activation_map - np.min(class_activation_map)
        class_activation_map = class_activation_map / np.max(class_activation_map)
        
        return weights, class_activation_map
        ### YOUR CODE ENDS HERE ###


if __name__ == '__main__':
    
    image = cv2.imread('cat_dog.png', 1)
    image = np.float32(cv2.resize(image, (224, 224))) / 255
    input = process_image(image)

    net = models.resnet34(pretrained=True)
    grad_cam = GradientCam(model=net, feature_module=net.layer4, \
                       target_layer_=["2"], cuda=False)
    

    target_index = None

    ### call to grad_cam method should return the importance vector corresponds to
    ### each feature map of last convolution layer of pretrained Resnet50

    weights, class_activation_map = grad_cam(input, target_index)

    print ("Most important feature map index: ", np.argmax(weights))
    print ("Its corresponding importance is: ", np.max(weights))

    visualize(image, class_activation_map)